In [85]:
# Import required packages
import numpy as np
import pandas as pd

import time 
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten, MaxPooling2D, Activation, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Nadam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from functools import partial
from pprint import pprint

from sklearn.datasets import load_boston
#from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import make_scorer, mean_squared_error, log_loss
from sklearn.model_selection import cross_val_score, KFold
from sklearn.utils import check_random_state

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [117]:
compas = pd.read_csv('../data/compas-scores-two-years.csv', encoding='latin-1')

In [119]:
compas.columns

Index(['sex', 'age', 'race', 'decile_score', 'priors_count', 'c_charge_degree',
       'two_year_recid', 'c_jail_in', 'c_jail_out'],
      dtype='object')

In [118]:
compas = compas[["sex","age","race","decile_score","priors_count","c_charge_degree","two_year_recid","c_jail_in", "c_jail_out"]]


In [50]:
compas = compas.loc[compas.race.isin(["Caucasian", "African-American"])]

In [44]:
compas.head()

,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_charge_degree,two_year_recid
1,1,25 - 45,African-American,0,3,0,0,0,F,1
2,1,Less than 25,African-American,0,4,0,1,4,F,1
3,1,Less than 25,African-American,0,8,1,0,1,F,0
6,1,25 - 45,Caucasian,0,6,0,0,14,F,1
8,0,25 - 45,Caucasian,0,1,0,0,0,M,0


In [120]:
compas_encoded = compas.copy()

In [162]:
compas_encoded = compas.copy()
compas_encoded.sex = pd.get_dummies(compas["sex"])["Female"]
compas_encoded.race = pd.get_dummies(compas["race"])["African-American"]
compas_encoded.c_charge_degree = pd.get_dummies(compas["c_charge_degree"])["F"]


compas_encoded['c_jail_in'] = pd.to_datetime(compas_encoded['c_jail_in'])
compas_encoded['c_jail_out'] = pd.to_datetime(compas_encoded['c_jail_out'])
compas_encoded['los'] = np.log((compas_encoded['c_jail_out']-compas_encoded['c_jail_in']).astype('timedelta64[h]'))#use log hours

compas_encoded.drop(["c_jail_in", "c_jail_out"], axis = 1, inplace=True)

compas_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
compas_encoded.dropna(inplace=True)

/opt/homebrew/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/homebrew/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [151]:
compas_encoded.head()

,sex,age,race,decile_score,priors_count,c_charge_degree,two_year_recid,los
0,0,69,0,1,0,1,0,3.135494
1,0,34,1,3,0,1,1,5.484797
2,0,24,1,4,4,1,1,3.258097
5,0,44,0,1,0,0,0,3.433987
6,0,41,0,6,14,1,1,5.017280


In [133]:
X = ['sex','age','decile_score','priors_count', 'los']
S = "race"
E = "c_charge_degree"
Y = "two_year_recid"
X_ALL = ['sex','age','decile_score','priors_count','race','c_charge_degree','los']


In [163]:
# 5:1:1 split
np.random.seed(5243)
train, test = train_test_split(compas_encoded, test_size=1/7)
train, val = train_test_split(train, test_size=1/6)


In [164]:
print(train.shape, test.shape, val.shape)

(4757, 8) (952, 8) (952, 8)


In [165]:
X_train = train[X_ALL]
L_train = train[X]
s_train = train[S]
e_train = train[E]
y_train = train[Y]

X_val = val[X_ALL]
L_val = val[X]
s_val = val[S]
e_val = val[E]
y_val = val[Y]

X_test = test[X_ALL]
L_test = test[X]
s_test = test[S]
e_test = test[E]
y_test = test[Y]

In [167]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_val, y_val)


0.6890756302521008

## Create functions used in pseudocode

In [168]:
# Creates a list of partitions - 1 for each unique value of e

# X is the full dataset (in our case, train)
# e is the 
def PARTITION(X):
    partitions = list()
    
    for e_i in np.unique(X[E]):
        partitions.append(X[X[E]==e_i])
    
    return partitions

In [169]:
def DELTA(X, X_ei, s_i):
    Gi = sum(X_ei[S] == s_i)
    
    X_ei_si = X_ei[X_ei[S] == s_i]
    
    P_denom = X_ei_si.shape[0]
    P_num = sum(X_ei_si[Y] == 1)
    P = P_num/P_denom
    
    X_ei_not_si = X_ei[X_ei[S] != s_i]
    
    Ps_2 = sum(X_ei_not_si[Y] == 1)/X_ei_not_si.shape[0]
    
    Ps = (P+Ps_2)/2
    
    d = int(round(Gi * abs(P - Ps)))
    
    return(d)

In [99]:
for X_ei in PARTITION(train):
    ranker_model = LogisticRegression(random_state=0).fit(X_ei[X], X_ei[Y])
    

In [110]:
ranker_model.decision_function(PARTITION(train)[0][PARTITION(train)[0][S]==1][X])

array([-0.00498393,  0.65107989, -1.15809679, -0.57008951, -0.97950422,
       -0.17787313,  0.17335436,  1.0122722 , -0.80631784,  0.2598263 ,
        0.10963466, -0.63313146, -1.15809679,  1.92622488, -1.41159187,
       -1.1798377 , -1.09324451, -0.15613221, -1.49818506,  0.10875635,
       -0.43707158, -0.38390997, -1.1798377 , -1.1798377 , -0.09069881,
        0.65107989, -0.63313146,  0.96953636, -1.15809679,  0.4463637 ,
        0.04420126,  1.59407038,  0.13079446,  0.70179429,  0.96904669,
       -1.00665132, -0.97950422, -0.17817032,  0.34138883,  0.91038745,
       -1.23840549,  0.48742812, -1.49818506, -0.29678297, -1.41159187,
        0.56916659, -0.60598436, -0.89291103, -0.95147882, -0.3782066 ,
        1.01974503,  1.48830865, -0.17817032, -0.49224408,  0.39598021,
        0.20050146, -1.15809679, -1.1798377 , -0.37850379, -0.80631784,
        0.34138883,  1.05612955, -1.1246652 , -0.3233313 , -0.06933089,
        0.31783759, -0.38818357,  0.24453474,  1.02412773,  1.71

In [111]:
ranker_model.predict(PARTITION(train)[0][PARTITION(train)[0][S]==1][X])

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,

In [112]:
ranker_model.predict_proba(PARTITION(train)[0][PARTITION(train)[0][S]==1][X])

array([[0.50124598, 0.49875402],
       [0.34274623, 0.65725377],
       [0.76098672, 0.23901328],
       ...,
       [0.62063494, 0.37936506],
       [0.13754671, 0.86245329],
       [0.58569695, 0.41430305]])